<a href="https://colab.research.google.com/github/benvictoria21/pandas/blob/master/Data_Cleaning_and_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 
from itertools import combinations
from collections import Counter
import warnings 

warnings.filterwarnings("ignore")
sns.set_style('darkgrid')
sns.color_palette("rocket")
%matplotlib inline

pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/chawla201/Kaggle-ML-DS-Survey-2020-Analysis/master/data/kaggle_survey_2020_responses.csv")

In [5]:
df.head()

,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,Q7_Part_4,Q7_Part_5,Q7_Part_6,Q7_Part_7,Q7_Part_8,Q7_Part_9,Q7_Part_10,Q7_Part_11,Q7_Part_12,Q7_OTHER,Q8,Q9_Part_1,Q9_Part_2,Q9_Part_3,Q9_Part_4,Q9_Part_5,Q9_Part_6,Q9_Part_7,Q9_Part_8,Q9_Part_9,Q9_Part_10,Q9_Part_11,Q9_OTHER,Q10_Part_1,Q10_Part_2,Q10_Part_3,Q10_Part_4,Q10_Part_5,Q10_Part_6,Q10_Part_7,...,Q31_B_Part_7,Q31_B_Part_8,Q31_B_Part_9,Q31_B_Part_10,Q31_B_Part_11,Q31_B_Part_12,Q31_B_Part_13,Q31_B_Part_14,Q31_B_OTHER,Q33_B_Part_1,Q33_B_Part_2,Q33_B_Part_3,Q33_B_Part_4,Q33_B_Part_5,Q33_B_Part_6,Q33_B_Part_7,Q33_B_OTHER,Q34_B_Part_1,Q34_B_Part_2,Q34_B_Part_3,Q34_B_Part_4,Q34_B_Part_5,Q34_B_Part_6,Q34_B_Part_7,Q34_B_Part_8,Q34_B_Part_9,Q34_B_Part_10,Q34_B_Part_11,Q34_B_OTHER,Q35_B_Part_1,Q35_B_Part_2,Q35_B_Part_3,Q35_B_Part_4,Q35_B_Part_5,Q35_B_Part_6,Q35_B_Part_7,Q35_B_Part_8,Q35_B_Part_9,Q35_B_Part_10,Q35_B_OTHER
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,In which country do you currently reside?,What is the highest level of formal education that you have attained or plan to attain within the next 2 years?,Select the title most similar to your current role (or most recent title if retired): - Selected Choice,For how many years have you been writing code and/or programming?,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - R,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - SQL,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C++,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Java,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Javascript,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Julia,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Swift,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Bash,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - MATLAB,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - None,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Other,What programming language would you recommend an aspiring data scientist to learn first? - Selected Choice,"Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - Jupyter (JupyterLab, Jupyter Notebooks, etc)",Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - RStudio,Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - Visual Studio / Visual Studio Code,Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - Click to write Choice 13,Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - PyCharm,Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - Spyder,Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - Notepad++,Which of the following integrate

In [9]:
questions = df.loc[:0]

questions

,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,Q7_Part_4,Q7_Part_5,Q7_Part_6,Q7_Part_7,Q7_Part_8,Q7_Part_9,Q7_Part_10,Q7_Part_11,Q7_Part_12,Q7_OTHER,Q8,Q9_Part_1,Q9_Part_2,Q9_Part_3,Q9_Part_4,Q9_Part_5,Q9_Part_6,Q9_Part_7,Q9_Part_8,Q9_Part_9,Q9_Part_10,Q9_Part_11,Q9_OTHER,Q10_Part_1,Q10_Part_2,Q10_Part_3,Q10_Part_4,Q10_Part_5,Q10_Part_6,Q10_Part_7,...,Q31_B_Part_7,Q31_B_Part_8,Q31_B_Part_9,Q31_B_Part_10,Q31_B_Part_11,Q31_B_Part_12,Q31_B_Part_13,Q31_B_Part_14,Q31_B_OTHER,Q33_B_Part_1,Q33_B_Part_2,Q33_B_Part_3,Q33_B_Part_4,Q33_B_Part_5,Q33_B_Part_6,Q33_B_Part_7,Q33_B_OTHER,Q34_B_Part_1,Q34_B_Part_2,Q34_B_Part_3,Q34_B_Part_4,Q34_B_Part_5,Q34_B_Part_6,Q34_B_Part_7,Q34_B_Part_8,Q34_B_Part_9,Q34_B_Part_10,Q34_B_Part_11,Q34_B_OTHER,Q35_B_Part_1,Q35_B_Part_2,Q35_B_Part_3,Q35_B_Part_4,Q35_B_Part_5,Q35_B_Part_6,Q35_B_Part_7,Q35_B_Part_8,Q35_B_Part_9,Q35_B_Part_10,Q35_B_OTHER
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,In which country do you currently reside?,What is the highest level of formal education that you have attained or plan to attain within the next 2 years?,Select the title most similar to your current role (or most recent title if retired): - Selected Choice,For how many years have you been writing code and/or programming?,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - R,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - SQL,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - C++,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Java,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Javascript,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Julia,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Swift,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Bash,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - MATLAB,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - None,What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Other,What programming language would you recommend an aspiring data scientist to learn first? - Selected Choice,"Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - Jupyter (JupyterLab, Jupyter Notebooks, etc)",Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - RStudio,Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - Visual Studio / Visual Studio Code,Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - Click to write Choice 13,Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - PyCharm,Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - Spyder,Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - Notepad++,Which of the following integrate

In [10]:
multiple_options_questions = set()
multiple_options_questions_A_B = set()

for q in list(questions.filter(regex=("Q\d+_Part_\d+")).columns):
    multiple_options_questions.add(int(q.split("_")[0][1:]))
for q in list(questions.filter(regex=("Q\d+_A_Part_\d+")).columns):
    multiple_options_questions_A_B.add(int(q.split("_")[0][1:]))

In [11]:
def extract_question(row):
    row = list(row)
    return row[0].split("-")[0].strip()

for i in list(multiple_options_questions):
    questions[f"Q{i}"] = questions.filter(regex=(f"Q{i}_Part_\d+")).apply(lambda row: extract_question(row.values.astype(str)), axis=1)
    
for i in list(multiple_options_questions_A_B):
    questions[f"Q{i}_A"] = questions.filter(regex=(f"Q{i}_A_Part_\d+")).apply(lambda row: extract_question(row.values.astype(str)), axis=1)
    questions[f"Q{i}_B"] = questions.filter(regex=(f"Q{i}_B_Part_\d+")).apply(lambda row: extract_question(row.values.astype(str)), axis=1)

In [13]:
questions.drop(columns=questions.filter(regex=("Q\d+_Part_\d+")).columns, inplace=True)
questions.drop(columns=questions.filter(regex=("Q\d+_A_Part_\d+")).columns, inplace=True)
questions.drop(columns=questions.filter(regex=("Q\d+_B_Part_\d+")).columns, inplace=True)
questions.drop(columns=questions.filter(regex=("Q\d+_A_OTHER")).columns, inplace=True)
questions.drop(columns=questions.filter(regex=("Q\d+_B_OTHER")).columns, inplace=True)
questions.drop(columns=questions.filter(regex=("Q\d+_OTHER")).columns, inplace=True)

In [14]:
questions= questions.transpose().iloc[1:]
questions.columns = ['Question']
questions.sort_index(inplace=True)
questions

,Question
Q1,What is your age (# years)?
Q10,Which of the following hosted notebook products do you use on a regular basis? (Select all that apply)
Q11,What type of computing platform do you use most often for your data science projects? - Selected Choice
Q12,Which types of specialized hardware do you use on a regular basis? (Select all that apply)
Q13,Approximately how many times have you used a TPU (tensor processing unit)?
Q14,What data visualization libraries or tools do you use on a regular basis? (Select all that apply)
Q15,For how many years have you used machine learning methods?
Q16,Which of the following machine learning frameworks do you use on a regular basis? (Select all that apply)
Q17,Which of the following ML algorithms do you use on a regular basis? (Select all that apply):
Q18,Which categories of computer vision methods do you use on a regular basis? (Select all that apply)


In [15]:
df = df[1:]
df.reset_index(inplace=True)

In [16]:
df.replace(to_replace=np.nan, value="Not Specified", inplace=True)

In [17]:
def group(row):
    row = list(row)
    result = list()
    for val in row:
        if val == "Not Specified":
            pass
        else:
            result.append(val.strip())
    return ";".join(result)

responses = pd.DataFrame()
for i in range(1, 40):
    if i in list(multiple_options_questions):
        responses[f"Q{i}"] = df.filter(regex=(f"Q{i}_Part_\d+")).apply(lambda row: group(row.values.astype(str)), axis=1)
    elif i in list(multiple_options_questions_A_B):
        responses[f"Q{i}_A"] = df.filter(regex=(f"Q{i}_A_Part_\d+")).apply(lambda row: group(row.values.astype(str)), axis=1)
        responses[f"Q{i}_B"] = df.filter(regex=(f"Q{i}_B_Part_\d+")).apply(lambda row: group(row.values.astype(str)), axis=1)
    else:
        responses[f"Q{i}"] = df[f"Q{i}"]

In [18]:
non_prof_index = responses[(responses['Q5']=='Student') | 
                           (responses['Q5']=='Currently not employed') | 
                           (responses['Q5']=='Other') |  
                           (responses['Q5']=='Not Specified') |
                           (responses['Q25']=='$0 ($USD)')].index

type_of_job_role =  list()
for index, row in responses.iterrows():
    if index in list(non_prof_index):
        type_of_job_role.append("Non Professional")
    else:
        type_of_job_role.append("Professional")
responses['type of Job Role'] = type_of_job_role

In [19]:
non_professional = responses[responses['type of Job Role']=="Non Professional"]
professional = responses[responses['type of Job Role']=="Professional"]

In [20]:
non_professional.drop(columns=non_professional.filter(regex=("Q\d+_A")).columns, inplace=True)
professional.drop(columns=professional.filter(regex=("Q\d+_B")).columns, inplace=True)